# 2024-05-13

In [14]:
import numpy as np
import pandas as pd

In [15]:
train_df = pd.read_table("nlp-flask/data/ratings_train.txt")
test_df = pd.read_table("nlp-flask/data/ratings_test.txt")

In [16]:
# Null 및 숫자를 공백 문자로 변경
# 대부분의 문자 변경은 가능하면 re를 사용
# 가능하면 축약 리스트 활용: 열에는 잘됨
# 가능하면 for 를 쓰지 않는 방향으로 진행
# 정규표현식 전화번호, 우편번호, 이메일 알아둘 것! 자주 쓰이는 것들이다.

import re

train_df = train_df.fillna(" ")
train_df["document"] = train_df["document"].apply(lambda x: re.sub(r"\d+", " ", x)) # apply: 전체 행에 실행 -> for문보다 빠르다: 행 단위 apply 쓰는 것 기억, 정규식 사용 기억

test_df = test_df.fillna(" ")
test_df["document"] = test_df["document"].apply(lambda x: re.sub(r"\d+", " ", x)) # apply: 전체 행에 실행 -> for문보다 빠르다

# 한국어 데이터: 레이블이 안 주어지면 내가 다 달아야 한다
# 텍스트 데이터는 전처리할 것이 굉장히 많다.
# pip install konlpy (konlp 아님!!!)

In [17]:
from konlpy.tag import Okt  # Open Korea Text, Kkma(는 엘라스틱 서치에서 사용): 토크나이저 다 해봐야 한다 어떤게 잘될지 모르니. 윈도우에서는 konlpy 사용(버전 0.6.0 이어야 한다): 자바 없이 가능해서 이거 사용
okt = Okt()

In [18]:
def tw_tokenizer(text):
    tokenizer_ko = okt.morphs(text)
    return tokenizer_ko
# 함수 꼭 만들어야 한다 -> tokenizer에 넣어줘야 한다

In [19]:
tw_tokenizer("가난한 내가 나타샤를 사랑해서")

['가난한', '내', '가', '나타', '샤를', '사랑', '해서']

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    tokenizer=tw_tokenizer, 
    ngram_range=(1,2), # 단어 두개 연관해서 보겠다
    min_df=3, # 하나 또는 두개씩만 본다. 세개 미만이면 버리고 세개면 계산
    max_df=0.9 # 90% 같으면 같은거다. (0.8~0.9) 1 잡으면 다 분리되니 그러지 마라
)

In [23]:
tfidf.fit(train_df["document"])

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<function tw_tokenizer at 0x0000022798A49040>)

In [24]:
tfidf_matrix_train = tfidf.transform(train_df["document"]) # 데이터분석 하는 사람들은 파이썬 공부를 하드코어하게 하라: 전처리 잘할 수 있어야(주니어) -> 텍스트 처리 방법은 해둬야 힘들지 않다
tfidf_matrix_train

<150000x129276 sparse matrix of type '<class 'numpy.float64'>'
	with 3069141 stored elements in Compressed Sparse Row format>

In [25]:
# 텍스트 -> 희소행렬(TF-IDF) -> 학습

from sklearn.linear_model import LogisticRegression
lg_clf = LogisticRegression(C=3.5, random_state=42)
lg_clf.fit(tfidf_matrix_train, train_df["label"]) # lg_clf: 기울기와 절편값이 잔뜩 들어가있다

LogisticRegression(C=3.5, random_state=42)

In [26]:
lg_clf.coef_

array([[3.67650441, 1.49510745, 0.2401038 , ..., 0.26156221, 1.08095266,
        0.28057158]])

In [27]:
import joblib # 생성한 행렬이 너무 커서 메모리에 올릴 수 없으므로 이것 사용

In [28]:
joblib.dump(lg_clf, "movie_lr.pkl")

['movie_lr.pkl']

In [29]:
joblib.dump(tfidf, "movie_lr_tfidf.pkl")

['movie_lr_tfidf.pkl']

# 역직렬화

In [30]:
movie_lr = joblib.load("movie_lr.pkl")
movie_lr_tfidf = joblib.load("movie_lr_tfidf.pkl")

In [33]:
q = "우와 재미있어요"
q = re.sub(r"\d+", " ", q) # 앞의 전처리를 복원해야 한다
q_matrix = movie_lr_tfidf.transform([q])

In [34]:
movie_lr.predict(q_matrix) # 1: 긍정

array([1], dtype=int64)

In [35]:
q = "에에에에에"
q = re.sub(r"\d+", " ", q) # 앞의 전처리를 복원해야 한다
q_matrix = movie_lr_tfidf.transform([q])

In [36]:
movie_lr.predict(q_matrix) # 0: 부정

array([0], dtype=int64)

In [ ]:
# 정규식 사용해야: 자바, 파이썬, 자바스크립트에서 돌아간다